### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Retinoblastoma/GSE63529'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely to contain gene expression data
if '!Series_title' in background_info and 'gene' in background_info['!Series_title'][0]:
    is_gene_available = True

# Data availability analysis
sample_characteristics = {0: ['cell line/subline: HEY', 'cell line/subline: HEY_SNS032-resistant', 'cell line/subline: HEY_PD0332991-resistant', 'cell line/subline: HEY_PD0332991/SNS032-resistant', 'cell line/subline: OAW28_parental', 'cell line/subline: OAW28_PD0332991/SNS032-resistant', 'cell line/subline: OV90_parental', 'cell line/subline: OAW28_SNS032-resistant', 'cell line/subline: OV90_PD0332991/SNS032-resistant', 'cell line/subline: OV90_SNS032-resistant', 'cell line/subline: SKOV3_parental', 'cell line/subline: SKOV3_PD0332991/SNS032-resistant_late', 'cell line/subline: SKOV3_SNS032-resistant', 'cell line/subline: SKOV3_PD0332991/SNS032-resistant_late_CDKi release', 'cell line/subline: SKOV3_PD0332991/SNS032-resistant_early'], 1: ['cell type: Ovarian cancer cell line'], 2: ['cell subtype/phenotype: parental', 'cell subtype/phenotype: SNS032-resistant', 'cell subtype/phenotype: PD0332991-resistant', 'cell subtype/phenotype: PD0332991/SNS032-resistant'], 3: ['treated with: none (control)', 'treated with: SNS032', 'treated with: PD0332991', nan]}

# Retinoblastoma (trait) analysis
if 2 in sample_characteristics and len(set(sample_characteristics[2])) > 1:
    trait_row = 2

# Age analysis - Not available
age_row = None

# Gender analysis - Not available
gender_row = None

# Data type conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip()
    if 'parental' in value:
        return 0
    elif 'resistant' in value:
        return 1
    return None

# Save cohort information
save_cohort_info('GSE63529', './preprocessed/Retinoblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Retinoblastoma', trait_row, convert_trait, None, None, None, None)
    csv_path = './preprocessed/Retinoblastoma/trait_data/GSE63529.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
